## basic shopping Mall crawler (made by ASH)

### 1. 목적

본 크롤러는 ICT 프로젝트 인공지능 드레스룸의 phase1 의류 등록 서비스를 구축하는데 필요한
기본적인 의류 데이터를 크롤링 하기 위해 만들어 졌다.

### 2. 정보 저장 객체

##### Product

* 상품에 관한 기본정보를 저장하기 위해 만들어 졌다

1) 상품 topic? 에 관한 필드    
2) 상품 url 필드   
3) 상품 이름 필드   
4) 상품 id 필드   
5) 상품 brand 필드

##### Website

* 크롤링 대상이 되는 웹사이트 도메인 별로 필요한 정보를 저장하기 위함

1) 이름  
2) 도메인 주소  
3) 검색 주소   
4) 검색 결과 list에 대한 css selector   
5) 검색 결과 list의 url에 대한 css selector   
6) 검색 결과(상품)의 url이 절대경로인지 상대경로인지 에 대한 bool   
7) title, id, brand 의 css selector

In [2]:
class Product:
    '''
    쇼핑몰/제품상세 페이지 전체에 사용할 기반 클래스
    '''
    def __init__(self, topic, url, title, Id, brand):
        self.topic = topic
        self.url = url
        self.title = title
        self.id = Id
        self.brand = brand

    def print(self):
        '''
        출력 결과를 원하는 대로 바꿀 수 있는 함수
        '''
        print('Product found for topic {}'.format(self.topic))
        print('URL: {}'.format(self.url))
        print('Title: {}'.format(self.title))
        print('Product id: {}'.format(self.id))
        print('brand: {}'.format(self.id))


class Website:
    '''
    웹사이트 구조에 관한 정보를 저장할 클래스
    '''
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, idTag, brandTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl # 절대url True 아니면 False
        self.titleTag = titleTag
        self.idTag = idTag
        self.brandTag = brandTag


### 3. 기본 쇼핑몰 크롤러(제작중)

* 스크립트언어가 없는 사이트/ 크롤링 봇에 대한 제한이 없는 사이트에서 사용할 수 있는 beautiful soup 기반의 크롤러


##### 진행상황
* 현재 기능은 검색을 기반으로한 스크래핑 기능이 있다.
* 일단 이미지에 대한 크롤링 기능은 만들지 않음
* 연습용이기 때문에 한 사이트의 한번 검색에 대해 5번의 재귀 카운트를 삽입한 상태

##### 계획
* 링크를 따라 이동하는 기능 추가예정
* 각종 예외상황에 유연하게 반응하는 기능 추가예정
* 스크립트 언어가 있을때/ 크롤링 봇에 대한 제한이 있는 경우에 대비한 새로운 크롤링 봇 제작 예정
* db 추가 기능 및 중복/ 페이지 골격 추적 기능 추가예정

In [3]:
import requests
from bs4 import BeautifulSoup

class ShopCrawler:
    
    def getPage(self, url):
        try :
            req = requests.get(url)
        except requests.exceptions.RequestException :
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''

    def search(self, topic, site):
        '''
        주어진 검색어로 주어진 웹사이트를 검색해 결과 페이지를 모두 기록합니다.
        '''
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        cnt = 0 #  <-- 반복 카운트 삽입 -->
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href']
            if site.absoluteUrl:
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url + url)
            if bs is None:
                print('Something was wrong with that page or URL. skipping!')
                return
            title = self.safeGet(bs, site.titleTag)
            # print('title:', title)
            Id = self.safeGet(bs, site.idTag)
            # print('Product id :', Id)
            brand = self.safeGet(bs, site.brandTag)
            # print('Product brand : ',brand)
            if title != '' and Id != '' and brand != '':
                product = Product(topic, url, title, Id, brand)
                product.print()
                print('\n')
            # <-- 반복 카운트 삽입 -->
            cnt += 1
            if cnt > 5:
                break


In [4]:
crawler = ShopCrawler()
siteData = [
    ['Musinsa Store', # name 
    'https://store.musinsa.com/', #url
    'https://search.musinsa.com/search/musinsa/?q=', #searchUrl
    'div.list_img', # resultListing
    'a', # resultURl
    True, # absoluteUrl
    'span.product_title_eng', # title
    'p.product_article_contents strong', # id
    'p.product_article_contents a', # brand
    ],
]
sites = []
for row in siteData:
    sites.append(Website(*row))

topics = ['shirt']


for topic in topics:
    print('Getting INFO ABOUT: ' + topic)
    for targetSite in sites:
        crawler.search(topic, targetSite)


Getting INFO ABOUT: shirt
Product found for topic shirt
URL: https://store.musinsa.com/app/product/detail/1208839/0
Title: [패키지] NYC LOCATION SWEAT SHIRTS [GMT521I13]
Product id: GROOVE RHYME / GMT521I13
brand: GROOVE RHYME / GMT521I13


Product found for topic shirt
URL: https://store.musinsa.com/app/product/detail/956939/0
Title: 1960 sweatshirts navy
Product id: UNIFORM BRIDGE / UB-1104
brand: UNIFORM BRIDGE / UB-1104


Product found for topic shirt
URL: https://store.musinsa.com/app/product/detail/1130844/0
Title: [패키지] NYC LOCATION LONG SLEEVE T-SHIRTS [GLT623I13]
Product id: GROOVE RHYME / GLT623I13
brand: GROOVE RHYME / GLT623I13


Product found for topic shirt
URL: https://store.musinsa.com/app/product/detail/859956/0
Title: Layla endless love Fluff Mood Check shirt S24 Navy
Product id: DIAMOND LAYLA / 134
brand: DIAMOND LAYLA / 134


Product found for topic shirt
URL: https://store.musinsa.com/app/product/detail/890749/0
Title: Wooltan Check Shirts (Cream)
Product id: JHONNY W